Market Snapshot (what agents are allowed to see)

In [82]:
from dataclasses import dataclass
from typing import Optional, List, Tuple
from abc import ABC, abstractmethod
import random
import itertools
import heapq
import pandas as pd
import time

In [83]:
@dataclass(frozen=True)
class MarketSnapshot:
    time: int
    best_bid: Optional[float]
    best_ask: Optional[float]
    spread: Optional[float]
    mid_price: Optional[float]

    # Optional L2
    bid_levels: List[Tuple[float, int]] = None
    ask_levels: List[Tuple[float, int]] = None

Agent Base Class

This enforces polymorphism

In [84]:
class Agent(ABC):
    def __init__(self, agent_id: int, cash: float = 10_000, inventory: int = 0):
        self.id = agent_id
        self.cash = cash
        self.inventory = inventory

    @abstractmethod
    def get_action(self, snapshot):
        """
        Return:
            - Order object
            - or None (no action)
        """
        pass

Order Intent (what agents emit)

Agents should emit intent, not touch the book.

In [85]:
class Order:
    def __init__(self, order_id, side, price, qty, timestamp):
        self.id = order_id
        self.side = side        # 'buy' or 'sell'
        self.price = price
        self.qty = qty
        self.ts = timestamp

In [86]:
class OrderBook:
    def __init__(self):
        self.bids = []   # max-heap using negative prices
        self.asks = []   # min-heap
        self.trades = []
        self.tape = []
        self._seq = itertools.count()   # unique sequence number

    def add_order(self, order):
        seq = next(self._seq)
        if order.side == 'buy':
            # negative price → max-heap behavior
            heapq.heappush(self.bids, (-order.price, order.ts, seq, order))
        else:
            heapq.heappush(self.asks, (order.price, order.ts, seq, order))

        self.match()

    def match(self):
        while self.bids and self.asks:
            buy = self.bids[0][3]
            sell = self.asks[0][3]

            if buy.price < sell.price:
                break

            traded_qty = min(buy.qty, sell.qty)

            self.trades.append(
                (buy.id, sell.id, sell.price, traded_qty)
            )

            self.tape.append({
                "time": time.time(),
                "price": sell.price,
                "qty": traded_qty
            })


            buy.qty -= traded_qty
            sell.qty -= traded_qty

            if buy.qty == 0:
                heapq.heappop(self.bids)
            if sell.qty == 0:
                heapq.heappop(self.asks)

    def print_top5(self):
        print("\nBUY SIDE")
        for _, _, _, o in heapq.nsmallest(5, self.bids):
          print(o.price, o.qty)

        print("SELL SIDE")
        for _, _, _, o in heapq.nsmallest(5, self.asks):
          print(o.price, o.qty)

    def best_bid(self):
        return self.bids[0][3].price if self.bids else None

    def best_ask(self):
        return self.asks[0][3].price if self.asks else None

    def mid_price(self):
        if self.best_bid() is None or self.best_ask() is None:
            return None
        return (self.best_bid() + self.best_ask()) / 2

In [87]:
class RandomAgent(Agent):
    def get_action(self, snapshot):
        # 50/50 buy or sell
        side = random.choice(["buy", "sell"])

        # Budget / inventory constraints
        if side == "buy" and self.cash <= 0:
            return None
        if side == "sell" and self.inventory <= 0:
            return None

        # Reference price
        mid = snapshot.mid_price
        if mid is None:
            mid = 100  # bootstrap price

        # Random near-touch limit order
        price_offset = random.choice([-1, 0, 1])
        price = max(1, int(mid + price_offset))

        qty = random.randint(1, 5)

        return Order(
            order_id=None,          # engine can assign
            side=side,
            price=price,
            qty=qty,
            timestamp=snapshot.time
        )

Engine → Agent interaction

In [88]:
class MarketEngine:
    def __init__(self, order_book, agents):
        self.order_book = order_book
        self.agents = agents
        self.time = 0

        # Analytics hooks (already useful for Day 4/5)
        self.tape = []
        self.l1_snapshots = []

    def step(self):
        """
        One discrete event step:
        - advance time
        - build snapshot
        - query agents
        - process orders
        """
        self.time += 1

        # ---- Build L1 snapshot ----
        best_bid = self.order_book.best_bid()
        best_ask = self.order_book.best_ask()

        spread = None
        mid = None
        if best_bid is not None and best_ask is not None:
            spread = best_ask - best_bid
            mid = (best_bid + best_ask) / 2

        snapshot = MarketSnapshot(
            time=self.time,
            best_bid=best_bid,
            best_ask=best_ask,
            spread=spread,
            mid_price=mid
        )

        # ---- Agents act ----
        for agent in self.agents:
            order = agent.get_action(snapshot)
            if order is not None:
                self.submit_order(order)

        # ---- Collect trades (tape) ----
        for trade in self.order_book.trades:
            self.tape.append(trade)

        self.order_book.trades = []

        # ---- Record L1 snapshot ----
        if best_bid is not None and best_ask is not None:
            self.l1_snapshots.append({
                "time": self.time,
                "best_bid": best_bid,
                "best_ask": best_ask,
                "spread": spread,
                "mid_price": mid
            })

    def submit_order(self, order):
        """
        Single gateway into the order book.
        Agents NEVER touch the book directly.
        """
        self.order_book.add_order(order)

    def run(self, steps: int):
        for _ in range(steps):
            self.step()


Agents were implemented using an abstract base class enforcing a uniform get_action(snapshot) interface. Each agent receives an immutable market snapshot containing only L1/L2 information and emits order intents without direct access to the order book, ensuring strict separation between decision logic and market mechanics.

SIMULATION

In [89]:
book = OrderBook()
# Bootstrap liquidity
book.add_order(Order(0, "buy", 100, 10, 0))
book.add_order(Order(1, "sell", 100, 10, 0))

agents = [RandomAgent(i) for i in range(10)]

engine = MarketEngine(book, agents)
engine.run(steps=5000)

Streaming output truncated to the last 5000 lines.
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
order
o

In [90]:
tape_df = pd.DataFrame(book.tape)
print(tape_df.head())

           time  price  qty
0  1.767795e+09    100   10


In [91]:
print("Number of trades:", len(engine.tape))

Number of trades: 1
